In [83]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine

In [84]:
db_file = 'eval_data.db'
data_tbl = 'eval_data_tbl'
user_tbl = 'user_tbl'
df = pd.DataFrame([['Stephanie', 84, 183, 100],
                   ['Sara', 79, 186, 80],
                   ['David', 67, 158, 75],
                   ['Ian', 52, 155, 45]])
df.columns = ['name', 'moc', 'hra', 'attendance']
# df.loc['Average'] = df.mean(numeric_only=True, axis=0)
# #replace NaN with 'Average'
# df.loc[pd.isna(df['name']), 'name'] = 'Average'
#round the numeric columns to two decimal places
df.round(2)

,name,moc,hra,attendance
0,Stephanie,84,183,100
1,Sara,79,186,80
2,David,67,158,75
3,Ian,52,155,45


In [85]:
initial_user_lst = ['Nadia', 'Gina', 'Lisa', 'Michael']
user_lst = initial_user_lst + df['name'].to_list()
#sort the list in alphabetical order
user_lst.sort()
user_lst

#create the user table in the sqlite database
conn = sqlite3.connect(db_file)
cur = conn.cursor()
cur.execute(f"CREATE TABLE IF NOT EXISTS {user_tbl} (name TEXT)")
for name in user_lst:
    cur.execute(f"INSERT INTO {user_tbl} (name) VALUES (?)", (name,))
conn.commit()

OperationalError: table user_tbl already exists

In [ ]:
#replace the names in the df with the names in the user table
# df.head()
query = cur.execute(f"SELECT * From {user_tbl}")
cols = [column[0] for column in query.description]
users = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
users.index =users.index + 1
users
#create dictionary of user name and index value
user_dict = dict(zip(users['name'], users.index))
user_dict


{'David': 1,
 'Gina': 2,
 'Ian': 3,
 'Lisa': 4,
 'Michael': 5,
 'Nadia': 6,
 'Sara': 7,
 'Stephanie': 8}

In [ ]:
#use user_dict to create an id column
df['name'] = df['name'].map(user_dict)
df.head()

,name,moc,hra,attendance
0,7,84,183,100
1,6,79,186,80
2,0,67,158,75
3,2,52,155,45


In [81]:
engine = create_engine(f'sqlite:///{db_file}') # using relative path
df.to_sql('eval_data_tbl', con=engine, if_exists='append', index=False)

4

In [50]:
# df.loc['Average'] = df.mean(numeric_only=True, axis=0)
# #replace NaN with 'Average'
# df.loc[pd.isna(df['name']), 'name'] = 'Average'

df
# df.info()

,name,moc,hra,attendance
0,Stephanie,84.0,183.0,100.0
1,Sara,79.0,186.0,80.0
2,David,67.0,158.0,75.0
3,Ian,52.0,155.0,45.0
Average,Average,70.5,170.5,75.0


In [46]:
conn = sqlite3.connect(db_file)
cur = conn.cursor()
cur.execute("SELECT * FROM eval_data_tbl")
rows = cur.fetchall()

#convert rows to a dataframe
df = pd.DataFrame(rows)
df.columns = ['name', 'moc', 'hra', 'attendance']
df
# print(type(rows))

# for row in rows:
#     print(row)

OperationalError: no such table: eval_data_tbl

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        4 non-null      object
 1   moc         4 non-null      int64 
 2   hra         4 non-null      int64 
 3   attendance  4 non-null      int64 
dtypes: int64(3), object(1)
memory usage: 256.0+ bytes


In [15]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Exception as e:
        print(e)
    return conn

In [ ]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks")
    rows = cur.fetchall()

    for row in rows:
        print(row)